Set `colab` to `False` if the notebook is not running on Colab.

In [1]:
colab = False

In [2]:
if colab:
    ROOT = '/content/gdrive/My Drive/Colab/kaggle/'
else:
    ROOT = '../'

In [3]:
MODULE_PATH = ROOT
DATA_PATH = ROOT + 'data/'
NOTEBOOK_PATH = ROOT + 'notebooks/'
RESULTS_PATH = ROOT + 'results/'

## Access to Drive

In [4]:
if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')

## Installations

In [5]:
if colab:
    get_ipython().system_raw('pip install torch torchvision pillow==4.1.1')

In [6]:
if colab:
    get_ipython().system_raw('mkdir data')
    get_ipython().system_raw('unzip "{path}" -d data'.format(path=DATA_PATH + 'trainset.zip'))
    DATA_PATH = 'data/trainset/'

# Import statements

## Access to modules

In [7]:
import sys
import os

In [8]:
sys.path.append(os.path.abspath(MODULE_PATH))

## Importation of modules

In [10]:
# "Magic" commands for automatic reloading of module, perfect for prototyping
%reload_ext autoreload
%autoreload 2

from models import wasserstein, variational

from torchvision.datasets import MNIST
from torchvision import transforms

from torch.utils.data import DataLoader, Subset

import torch.optim as optim
import torch

from tqdm import tqdm

import numpy as np

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset

In [12]:
dataset = MNIST(DATA_PATH, download=True, transform=transforms.ToTensor())

In [13]:
transforms.functional.to_pil_image(dataset[0][0])

In [14]:
split = .9

n = len(dataset)
indices = list(range(n))

np.random.seed(0)
np.random.shuffle(indices)

train_indices = indices[:int(split * n)]
valid_indices = indices[int(split * n):]

train_set = Subset(dataset, train_indices)
valid_set = Subset(dataset, valid_indices)

In [15]:
train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=100, shuffle=True)

# Training

## Variational

### Defining the model

In [28]:
model = variational.VariationalAutoEncoder()
model

VariationalAutoEncoder(
  (reconstruction_criterion): MSELoss()
  (activation): ReLU()
  (dropout): Dropout(p=0.1)
  (max): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder_conv32): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_conv16): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_conv8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_lin_100): Linear(in_features=12544, out_features=100, bias=True)
  (encoder_lin_mu): Linear(in_features=100, out_features=2, bias=True)
  (encoder_lin_sigma): Linear(in_features=100, out_features=2, bias=True)
  (decoder_lin_l): Linear(in_features=2, out_features=100, bias=True)
  (decoder_lin_100): Linear(in_features=100, out_features=12544, bias=True)
  (decoder_conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 

In [29]:
model = model.to(device)

### Custom train and test functions

In [30]:
def train(epoch, model, optimizer, dataloader, device):

    model.train()
    train_loss = 0
    
    iterator = tqdm(
        dataloader,
        ascii=True,
        ncols=100,
        total=len(dataloader)
    )

    for data, _ in iterator:
        # We move the mini-batch to the device (useful is using a GPU)
        data = data.to(device)

        # We initialize the gradients
        optimizer.zero_grad()

        # We compute the recontruction of x (x_tilde) and its encoding (z)
        x_tilde, mu, log_sigma = model(data)

        # We compute the loss
        loss = model.loss(x_tilde=x_tilde, x=data, mu=mu, log_sigma=log_sigma, device=device)

        # Backpropagation
        loss.backward()

        # Updating the loss
        train_loss += loss.item()

        # Updating the parameters
        optimizer.step()

    print('>> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [31]:
def test(epoch, model, dataloader, device, writer):

    model.eval()
    test_loss = 0

    # We do not compute gradients during the testing phase, hence the no_grad() environment
    with torch.no_grad():

        for i, (data, _) in enumerate(dataloader):

            data = data.to(device)
            
            x_tilde, mu, log_sigma = model(data)

            # We compute the loss
            loss = model.loss(x_tilde=x_tilde, x=data, mu=mu, log_sigma=log_sigma, device=device)

            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n], x_tilde.view(100, 1, 28, 28)[:n]])

    test_loss /= len(test_loader.dataset)
    print('>> Test set loss: {:.4f}'.format(test_loss))

### Training loop

In [32]:
optimizer = optim.Adam(model.parameters())

In [33]:
train(1, model, optimizer, train_loader, device)


  3%|#7                                                            | 15/540 [00:24<16:37,  1.90s/it]

KeyboardInterrupt: 

## Wasserstein

### Defining the model

In [20]:
model = wasserstein.WassersteinAutoEncoder()
model

WassersteinAutoEncoder(
  (reconstruction_criterion): MSELoss()
  (activation): ReLU()
  (dropout): Dropout(p=0.1)
  (max): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder_conv32): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_conv16): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_conv8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (encoder_lin_100): Linear(in_features=4096, out_features=100, bias=True)
  (encoder_lin_l): Linear(in_features=100, out_features=2, bias=True)
  (decoder_lin_l): Linear(in_features=2, out_features=100, bias=True)
  (decoder_lin_100): Linear(in_features=100, out_features=4096, bias=True)
  (decoder_conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (decoder_conv8): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding

In [17]:
model = model.to(device)

### Custom train and test functions

In [18]:
def train(epoch, model, optimizer, dataloader, device):

    model.train()
    train_loss = 0
    
    iterator = tqdm(
        dataloader,
        ascii=True,
        ncols=100,
        total=len(dataloader)
    )

    for data, _ in iterator:
        # We move the mini-batch to the device (useful is using a GPU)
        data = data.to(device)

        # We initialize the gradients
        optimizer.zero_grad()

        # We compute the recontruction of x (x_tilde) and its encoding (z)
        x_tilde, z = model(data)

        # We compute the loss
        loss = model.loss(x_tilde=x_tilde, x=data, z=z, device=device)

        # Backpropagation
        loss.backward()

        # Updating the loss
        train_loss += loss.item()

        # Updating the parameters
        optimizer.step()

    print('>> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [19]:
def test(epoch, model, dataloader, device, writer):

    model.eval()
    test_loss = 0

    # We do not compute gradients during the testing phase, hence the no_grad() environment
    with torch.no_grad():

        for i, (data, _) in enumerate(dataloader):

            data = data.to(device)
            x_tilde, z = model(data)

            test_loss += model.loss(x_tilde=x_tilde, x=data, z=z, device=device).item()

            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n], x_tilde.view(100, 1, 28, 28)[:n]])

    test_loss /= len(test_loader.dataset)
    print('>> Test set loss: {:.4f}'.format(test_loss))

### Training loop

In [19]:
optimizer = optim.Adam(model.parameters())

In [ ]:
train(1, model, optimizer, train_loader, device)

  0%|                                                                       | 0/540 [00:00<?, ?it/s]/Users/basile/miniconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/Users/basile/miniconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 66%|########################################3                    | 357/540 [10:29<05:23,  1.77s/it]